## QIB Email *Extract, Load, and Transform* Notebook

### Imports

In [1]:
# Standard Library Imports
import logging
from typing import List, Dict, Any
from datetime import datetime
import time

# Third-Party Imports
import pypff # type: ignore
from sqlalchemy.exc import SQLAlchemyError
from tqdm.notebook import tqdm

# Local Imports
from src.config.config import Config
from src.extract.message_extractor import PstMessageExtractor
from src.transform.message_parser import MessageParser, ParsedMessage
from src.transform.message_enricher import MessageEnricher, EnrichedMessage
from src.load.data_loader import DataLoader
from src.database.database import Database
from src.database.export_utils import DataExporter

logging.basicConfig(level=logging.INFO)
logging.getLogger('chardet.charsetprober').disabled = True

### Configuration

In [2]:
config = Config.from_json("config.json")
extractor = PstMessageExtractor(config.input_pst_path, config.chunk_size)
message_parser = MessageParser()
message_enricher = MessageEnricher()
database = Database.from_credentials(username=config.db_user, password=config.db_password, host=config.db_host, database=config.db_name)
loader = DataLoader(database)
exporter = DataExporter(database)

### ETL Pipeline with Error Handling

In [3]:
total_messages = extractor.get_total_messages("Inbox")
pbar = tqdm(total=total_messages, desc='Extracting messages')
start_time = time.time()

database.drop_all_tables()
loader.create_tables()
logging.info("Starting ETL Pipeline")
for message_batch in extractor.extract_messages("Inbox"):
    total_messages_in_batch = len(message_batch.messages)
    enriched_messages: List[EnrichedMessage] = []
    loaded_messages: int = 0
    
    for pst_message in message_batch.messages:
        try:
            parsed_message: ParsedMessage = message_parser.parse(pst_message.message, pst_message.folder_name)
            enriched_message: EnrichedMessage = message_enricher.enrich_message(parsed_message)
            loader.load(enriched_message)
        except SQLAlchemyError as e:
            logging.error(f"Error loading message {enriched_message.provider_email_id}: {e}")
            continue
        except Exception as e:
            continue
        else:
            enriched_messages.append(enriched_message)
        finally:
            loaded_messages += 1
            pbar.update(1)

end_time = time.time()
pbar.close()
logging.info(f"Finished ETL Pipeline in {int(end_time - start_time) // 60} minutes and {int(end_time - start_time) % 60} seconds")

Extracting messages:   0%|          | 0/5039 [00:00<?, ?it/s]

INFO:root:All tables have been dropped from the database.
INFO:root:All tables have been created in the database.
INFO:root:Starting ETL Pipeline


internal_communication 


KeyboardInterrupt: 

In [ ]:
exporter.export_to_excel("./data/processed")

In [ ]:
exporter.export_schema('./data')